In [3]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display

## Семинар

Написать краулер, который собирает ссылки со страниц румынской Википедии.

In [2]:
url = 'http://mo.wikipedia.org' ## можно только страницы, которые ссылаются на вики

In [3]:
def parse_mo_wiki(url):
    to_visit = set()  # это чтобы в очередь не добавлять одну и ту же ссылку по несколько раз
    visited_links = set()
    queue = [url]
    while queue:
        link = queue.pop()
        visited_links.add(link)
        req = requests.get(link)
        soup = BeautifulSoup(req.text, 'lxml')
        for i in soup.findAll('a', href = True):
            if i['href'].startswith('/wiki/'):
                if ':' not in i['href']:  # это чтобы убрать служебные статьи
                    if i['href'] not in visited_links and i['href'] not in to_visit:
                        to_visit.add(i['href'])
                        queue.append('http://mo.wikipedia.org'+i['href'])
    return visited_links

In [4]:
all_links = parse_mo_wiki(url)

In [5]:
len(all_links)

320

## ДЗ 

Написать краулер, который собирает тексты с новостного ресурса.

In [1]:
kaz_url = 'http://www.evening-kazan.ru'

In [4]:
req = requests.get(kaz_url)

In [5]:
soup = BeautifulSoup(req.text, 'lxml')

In [6]:
main_links = []
for i in soup.findAll('li',attrs = {'class':['leaf', 'leaf first']}):
    if 'categories' in i.contents[0]['href']:
        main_links.append(('http://www.evening-kazan.ru'+ i.contents[0]['href']+'?page=', i.text))

In [8]:
main_links

[('http://www.evening-kazan.ru/categories/politika.html?page=', 'Власть'),
 ('http://www.evening-kazan.ru/categories/ekonomika.html?page=', 'Экономика'),
 ('http://www.evening-kazan.ru/categories/medicina.html?page=', 'Медицина'),
 ('http://www.evening-kazan.ru/categories/socproblemy.html?page=',
  'Соц.проблемы и ЖКХ'),
 ('http://www.evening-kazan.ru/categories/sreda-obitaniya.html?page=',
  'Среда обитания'),
 ('http://www.evening-kazan.ru/categories/kriminal.html?page=',
  'Криминал и ЧП'),
 ('http://www.evening-kazan.ru/categories/obrazovanie.html?page=',
  'Образование'),
 ('http://www.evening-kazan.ru/categories/kultura.html?page=', 'Культура'),
 ('http://www.evening-kazan.ru/categories/obshchestvo.html?page=', 'Общество'),
 ('http://www.evening-kazan.ru/categories/sport.html-0?page=', 'Спорт'),
 ('http://www.evening-kazan.ru/categories/abzac-inform.html?page=',
  'Абзац-информ'),
 ('http://www.evening-kazan.ru/categories/zvyozdy-ne-vrut.html?page=',
  'Звёзды не врут'),
 ('http:

In [9]:
count_all = 0
links_from_class = set()
for class_link in main_links:
    topic = class_link[1]
    for i in range(4):
        try:
            req = requests.get(class_link[0]+str(i))
            soup = BeautifulSoup(req.text, 'lxml')
            for i in soup.findAll('a', href=True):
                if 'http://www.evening-kazan.ru/articles' in i['href'] and '#disqus_thread' not in i['href'] and '#comment'not in i['href']:
                    links_from_class.add((i['href'], topic))
                    count_all+=1
        except:
            break

" тут мы получили ссылки на чуть более чем 1к статей"

' тут мы получили ссылки на чуть более чем 1к статей'

In [20]:
#get_texts_and_info

index = 1
for link in links_from_class:
    try:
        req = requests.get(link[0])
        soup = BeautifulSoup(req.text, 'lxml')
        author = soup.findAll('div', attrs = {'class':'author heading--meta'})[0].text
        title = soup.findAll('h1', attrs = {'class':'title title-story'})[0].text
        date = soup.findAll('div', attrs = {'submitted heading--meta'})[0].text
        text = soup.findAll('div', attrs = {'class':'node'})
        text_to_write = ''
        for i in text:
            for item in i.findAll('p'):
                text_to_write+=item.text
        with open(str(index)+'.txt', 'w') as file:
            file.write('@au '+author+'\n'+'@ti '+title+'\n'+'@da '+date+'\n'+'@topic '+link[1]+'\n'+'@url '+link[0]+'\n'+text_to_write)
        index+=1
    except Exception as e:
        print(e)